In [15]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from raise_utils.data import Data
from raise_utils.hyperparams import DODGE
from raise_utils.learners import *
from keras.utils.np_utils import to_categorical

In [21]:
def split_data(filename: str, data: Data, n_classes: int):
    if n_classes == 2:
        if filename == 'firefox.csv':
            data.y_train = data.y_train < 4
            data.y_test = data.y_test < 4
        elif filename == 'chromium.csv':
            data.y_train = data.y_train < 5
            data.y_test = data.y_test < 5
        else:
            data.y_train = data.y_train < 6
            data.y_test = data.y_test < 6
    elif n_classes == 3:
        data.y_train = np.where(data.y_train < 2, 0,
                                np.where(data.y_train < 6, 1, 2))
        data.y_test = np.where(
            data.y_test < 2, 0, np.where(data.y_test < 6, 1, 2))
    elif n_classes == 5:
        data.y_train = np.where(data.y_train < 1, 0, np.where(data.y_train < 3, 1, np.where(
            data.y_train < 6, 2, np.where(data.y_train < 21, 3, 4))))
        data.y_test = np.where(data.y_test < 1, 0, np.where(data.y_test < 3, 1, np.where(
            data.y_test < 6, 2, np.where(data.y_test < 21, 3, 4))))
    elif n_classes == 7:
        data.y_train = np.where(data.y_train < 1, 0, np.where(data.y_train < 2, 1, np.where(data.y_train < 3, 2, np.where(
            data.y_train < 6, 3, np.where(data.y_train < 11, 4, np.where(data.y_train < 21, 5, 6))))))
        data.y_test = np.where(data.y_test < 1, 0, np.where(data.y_test < 2, 1, np.where(data.y_test < 3, 2, np.where(
            data.y_test < 6, 3, np.where(data.y_test < 11, 4, np.where(data.y_test < 21, 5, 6))))))
    else:
        data.y_train = np.where(data.y_train < 1, 0, np.where(data.y_train < 2, 1, np.where(data.y_train < 3, 2, np.where(data.y_train < 4, 3, np.where(
            data.y_train < 6, 4, np.where(data.y_train < 8, 5, np.where(data.y_train < 11, 6, np.where(data.y_train < 21, 7, 8))))))))
        data.y_test = np.where(data.y_test < 1, 0, np.where(data.y_test < 2, 1, np.where(data.y_test < 3, 2, np.where(data.y_test < 4, 3, np.where(
            data.y_test < 6, 4, np.where(data.y_test < 8, 5, np.where(data.y_test < 11, 6, np.where(data.y_test < 21, 7, 8))))))))

    #if n_classes > 2:
    #    data.y_train = to_categorical(data.y_train, num_classes=n_classes)
    #    data.y_test = to_categorical(data.y_test, num_classes=n_classes)
    
    return data

In [24]:
def run(filename, n_classes):
    df = pd.read_csv(f'./Bug-Related-Activity-Logs/{filename}.csv')
    df.drop(['Unnamed: 0', 'bugID'], axis=1, inplace=True)
    _df = df[['s1', 's2', 's3', 's4', 's5', 's6', 's8', 'y']]
    _df['s70'] = df['s7'].apply(lambda x: eval(x)[0])
    _df['s71'] = df['s7'].apply(lambda x: eval(x)[1])
    _df['s72'] = df['s7'].apply(lambda x: eval(x)[2])
    _df['s90'] = df['s9'].apply(lambda x: eval(x)[0])
    _df['s91'] = df['s9'].apply(lambda x: eval(x)[1])
    
    if filename == 'firefox':
        _df['s92'] = df['s9'].apply(lambda x: eval(x)[2])
    
    x = _df.drop('y', axis=1)
    y = _df['y']
    
    data = Data(*train_test_split(x, y))
    data = split_data(filename, data, n_classes)
    
    config = {
        "n_runs": 20,
        "transforms": ["normalize", "standardize", "robust", "maxabs", "minmax"] * 30,
        "metrics": ["accuracy"],
        "random": True,
        "learners": [],
        "log_path": "./dodge-log/",
        "data": [data],
        "name": f"{filename}-{n_classes}"
    }
    for _ in range(50):
        config["learners"].extend([
            DecisionTree(random=True),
            RandomForest(random=True),
            LogisticRegressionClassifier(random=True),
            NaiveBayes(random=True)
        ])

    dodge = DODGE(config)
    dodge.optimize()

In [10]:
!mkdir dodge-log

mkdir: dodge-log: File exists


In [26]:
for file in ['firefox', 'eclipse', 'chromium']:
    print('Running:', file)
    print('=' * 30)
    for n_class in [2, 3, 5, 7, 9]:
        if os.path.exists(f'./dodge-log/{file}-{n_class}.txt'):
            continue
        print(n_class, 'classes')
        print('=' * 30)
        run(file, n_class)

Running: firefox
Running: eclipse
5 classes
{'hooks': None, 'learner': DecisionTreeClassifier(criterion='entropy', splitter='random'), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random']}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': RandomForestClassifier(n_estimators=71), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'n_estimators': (10, 100)}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': LogisticRegression(C=1000.0, penalty='l1', solver='liblinear'), 'name': 'rf', 'random': True, 'random_map': {'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 10.0, 100.0, 1000.0]}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner'

{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': DecisionTreeClassifier(criterion='entropy'), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random']}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': RandomForestClassifier(n_estimators=28), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'n_estimators': (10, 100)}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': LogisticRegression(C=1000.0, solver='liblinear'), 'name': 'rf', 'random': True, 'random_map': {'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 10.0, 100.0, 1000.0]}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': Non

{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': DecisionTreeClassifier(criterion='entropy', splitter='random'), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random']}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': RandomForestClassifier(n_estimators=60), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'n_estimators': (10, 100)}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': LogisticRegression(C=0.1, penalty='l1', solver='liblinear'), 'name': 'rf', 'random': True, 'random_map': {'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 10.0, 100.0, 1000.0]}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, '

{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': DecisionTreeClassifier(), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random']}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': RandomForestClassifier(n_estimators=50), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'n_estimators': (10, 100)}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': LogisticRegression(C=1000.0, solver='liblinear'), 'name': 'rf', 'random': True, 'random_map': {'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 10.0, 100.0, 1000.0]}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': None, 'x_train': None,

{'hooks': None, 'learner': LogisticRegression(C=100.0, penalty='l1', solver='liblinear'), 'name': 'rf', 'random': True, 'random_map': {'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 10.0, 100.0, 1000.0]}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': DecisionTreeClassifier(criterion='entropy', splitter='random'), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random']}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': RandomForestClassifier(n_estimators=20), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'n_estimators': (10, 100)}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': LogisticRegression(C=100.0, solver='libline

...

{'hooks': None, 'learner': DecisionTreeClassifier(), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random']}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': RandomForestClassifier(n_estimators=72), 'name': 'rf', 'random': True, 'random_map': {'criterion': ['gini', 'entropy'], 'n_estimators': (10, 100)}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': LogisticRegression(C=0.1, penalty='l1', solver='liblinear'), 'name': 'rf', 'random': True, 'random_map': {'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 10.0, 100.0, 1000.0]}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': GaussianNB(), 'name': 'rf', 'random': True, 'random_map': {}, 'x_test': None, 'x_train': None, 'y_test': None, 'y_train': None}
{'hooks': None, 'learner': DecisionTreeClassifier(criterion='entropy'), 'name': 'rf', 'random': True, 'random_

setting: maxabsU|rf
iter 4: [0.15324838580652533]
setting: standardizeY|rf
iter 5: [0.1750757099594309]
setting: normalizeU|rf
iter 6: [0.12547854408319525]
setting: maxabsE|rf
iter 7: [0.14896291640477688]
setting: normalizeQ|rf
iter 8: [0.17701845608822353]
setting: standardizep|rf
iter 9: [0.17701845608822353]
setting: maxabsB|rf
iter 10: [0.17701845608822353]
setting: minmaxM|rf
iter 11: [0.14930575395691675]
setting: standardizew|rf
iter 12: [0.1547340151991315]
setting: robustj|rf
iter 13: [0.12496428775498543]
setting: normalizee|rf
iter 14: [0.15341980458259527]
setting: robustH|rf
iter 15: [0.14924861436489342]
setting: robustI|rf
iter 16: [0.12210730815381979]
setting: normalizeT|rf
iter 17: [0.1503914062053597]
setting: standardizeC|rf
iter 18: [0.1767898977201303]
setting: maxabsC|rf
iter 19: [0.14839152048454374]
setting: minmaxU|rf
iter 20: [0.1234786583623793]
setting: robustE|rf
iter 21: [0.15513399234329467]
setting: robustf|rf
iter 22: [0.1235357979544026]
setting: ro

setting: minmaxP|rf
iter 10: [0.1529626878464088]
setting: standardizeq|rf
iter 11: [0.14953431232501]
setting: standardizer|rf
iter 12: [0.17650419976001372]
setting: robustv|rf
iter 13: [0.17650419976001372]
setting: maxabsp|rf
iter 14: [0.12245014570595966]
setting: normalizeP|rf
iter 15: [0.17661847894406035]
setting: maxabsL|rf
iter 16: [0.1534769441746186]
setting: maxabsw|rf
iter 17: [0.15427689846294498]
setting: minmaxL|rf
iter 18: [0.14982001028512656]
setting: normalizeM|rf
iter 19: [0.1243928918347523]
setting: maxabsY|rf
iter 20: [0.17479001199931432]
setting: minmaxQ|rf
iter 21: [0.1477629849722873]
setting: standardizeg|rf
iter 22: [0.15279126907033883]
setting: minmaxy|rf
iter 23: [0.17479001199931432]
setting: robustm|rf
iter 24: [0.1543340380549683]
setting: normalizeU|rf
iter 25: [0.17587566424775727]
setting: maxabsa|rf
iter 26: [0.17587566424775727]
setting: maxabsc|rf
iter 27: [0.12387863550654249]
setting: minmaxj|rf
iter 28: [0.14964859150905663]
setting: normal

setting: standardizeT|rf
iter 17: [0.12279298325809954]
setting: minmaxK|rf
iter 18: [0.17581852465573397]
setting: maxabsk|rf
iter 19: [0.1539340609108051]
setting: maxabsy|rf
iter 20: [0.15490543397520143]
setting: standardizeL|rf
iter 21: [0.12285012285012285]
setting: normalizeO|rf
iter 22: [0.15313410662247873]
setting: normalizef|rf
iter 23: [0.175647105879664]
setting: standardized|rf
iter 24: [0.175647105879664]
setting: normalizee|rf
iter 25: [0.17570424547168734]
setting: normalizeu|rf
iter 26: [0.15273412947831552]
setting: minmaxj|rf
iter 27: [0.15444831723901492]
setting: standardizeu|rf
iter 28: [0.17473287240729102]
setting: normalizeI|rf
iter 29: [0.14902005599680018]
Run # 12
setting: standardizee|rf
iter 0: [0.1512485000857094]
setting: maxabsS|rf
iter 1: [0.12267870407405292]
setting: robustn|rf
iter 2: [0.1499914290611965]
setting: robustu|rf
iter 3: [0.14924861436489342]
setting: minmaxJ|rf
iter 4: [0.12267870407405292]
setting: normalizeI|rf
iter 5: [0.12222158733

setting: normalizeP|rf
iter 23: [0.15256271070224559]
setting: maxabsm|rf
iter 24: [0.15381978172675848]
setting: robusts|rf
iter 25: [0.1535340837666419]
setting: standardizer|rf
iter 26: [0.12307868121821611]
setting: minmaxg|rf
iter 27: [0.15341980458259527]
setting: standardizeJ|rf
iter 28: [0.14924861436489342]
setting: normalizeg|rf
iter 29: [0.12285012285012285]
Run # 17
setting: normalizeU|rf
iter 0: [0.15021998742928974]
setting: standardizev|rf
iter 1: [0.12227872692988973]
setting: normalizeV|rf
iter 2: [0.15364836295068854]
setting: maxabsY|rf
iter 3: [0.17770413119250328]
setting: robusta|rf
iter 4: [0.17770413119250328]
setting: minmaxq|rf
iter 5: [0.14902005599680018]
setting: maxabsg|rf
iter 6: [0.17770413119250328]
setting: normalizeg|rf
iter 7: [0.15033426661333638]
setting: normalizeq|rf
iter 8: [0.1503914062053597]
setting: standardizer|rf
iter 9: [0.147705845380264]
setting: minmaxY|rf
iter 10: [0.1235357979544026]
setting: normalizet|rf
iter 11: [0.177418433232386